<a href="https://colab.research.google.com/github/onlyabhilash/Transformers_with_NLP/blob/main/ProphetNet/ProphetNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Mount Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


# Set up repo

In [ ]:
cd drive/My Drive/Inter_IIT_Headline_Generation/HeadlineGen

/content/drive/My Drive/Inter_IIT_Headline_Generation/HeadlineGen


In [ ]:
#!mkdir models/

In [ ]:
cd models/

/content/drive/My Drive/Inter_IIT_Headline_Generation/HeadlineGen/models


In [ ]:
#!mkdir ProphetNet/

In [ ]:
cd ProphetNet

/content/drive/My Drive/Inter_IIT_Headline_Generation/HeadlineGen/models/ProphetNet


In [ ]:
#!mkdir saved_models/

# Pull from repo

In [ ]:
!git pull origin main

remote: Enumerating objects: 7, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 6 (delta 2), reused 6 (delta 2), pack-reused 0
Unpacking objects: 100% (6/6), done.
From https://github.com/atharva-naik/HeadlineGen
 * branch            main       -> FETCH_HEAD
   d8deece..2c5e4a4  main       -> origin/main
hint: Waiting for your editor to close the file... error: unable to start editor 'editor'
Not committing merge; use 'git commit' to complete the merge.


# Push to Repo

In [ ]:
uname = "vmm221313"
!git config --global user.email '$uname@gmail.com'
!git config --global user.name '$uname'

In [ ]:
from getpass import getpass
password = getpass('Password:')

In [ ]:
cd ../../../

/


In [ ]:
ls

bin/      datalab/  home/   lib64/  opt/   run/   sys/                tools/
boot/     dev/      lib/    media/  proc/  sbin/  tensorflow-1.15.2/  usr/
content/  etc/      lib32/  mnt/    root/  srv/   tmp/                var/


In [ ]:
cd content/drive/My Drive/Inter_IIT_Headline_Generation/HeadlineGen

/content/drive/My Drive/Inter_IIT_Headline_Generation/HeadlineGen


In [ ]:
!git status

On branch main
Your branch is up to date with 'origin/main'.

Changes not staged for commit:
  (use "git add/rm <file>..." to update what will be committed)
  (use "git checkout -- <file>..." to discard changes in working directory)

	deleted:    models/.gitignore

Untracked files:
  (use "git add <file>..." to include in what will be committed)

	LOG_FILE
	models/ProphetNet/

no changes added to commit (use "git add" and/or "git commit -a")


In [ ]:
!git add models/

In [ ]:
!git commit -m 'ProphetNet files'  

[main 3151f31] ProphetNet files
 12 files changed, 2 deletions(-)
 delete mode 100644 models/.gitignore
 create mode 100644 models/ProphetNet/logs/1615391470.3505876/events.out.tfevents.1615391470.4637b92c1487.1991.1
 create mode 100644 models/ProphetNet/logs/1615394327.1252856/events.out.tfevents.1615394327.4637b92c1487.2959.1
 create mode 100644 models/ProphetNet/logs/1615395638.5619366/events.out.tfevents.1615395638.4637b92c1487.2959.3
 create mode 100644 models/ProphetNet/logs/1615395823.60891/events.out.tfevents.1615395823.4637b92c1487.2959.5
 create mode 100644 models/ProphetNet/logs/1615395908.5650058/events.out.tfevents.1615395908.4637b92c1487.2959.7
 create mode 100644 models/ProphetNet/logs/events.out.tfevents.1615391470.4637b92c1487.1991.0
 create mode 100644 models/ProphetNet/logs/events.out.tfevents.1615394327.4637b92c1487.2959.0
 create mode 100644 models/ProphetNet/logs/events.out.tfevents.1615395638.4637b92c1487.2959.2
 create mode 100644 models/ProphetNet/logs/events.o

In [ ]:
!git push origin main     

Counting objects: 24, done.
Delta compression using up to 2 threads.
Compressing objects: 100% (22/22), done.
Writing objects: 100% (24/24), 42.93 KiB | 1.72 MiB/s, done.
Total 24 (delta 10), reused 0 (delta 0)
remote: Resolving deltas: 100% (10/10), completed with 1 local object.
To https://github.com/atharva-naik/HeadlineGen/
   2c5e4a4..87d82c9  main -> main


# Imports and Installs

In [ ]:
!pip install datasets
!pip install transformers

In [ ]:
!pip install git-python==1.0.3
!pip install sacrebleu==1.4.12
!pip install rouge_score

In [ ]:
import datasets
import pandas as po
from tqdm.notebook import tqdm

In [ ]:
import torch
import torch.nn as nn
from typing import Optional
from transformers import Trainer
from dataclasses import dataclass, field
from transformers import TrainingArguments
from torch.utils.data import Dataset, DataLoader, TensorDataset
from transformers import AdamW, get_linear_schedule_with_warmup

# ProphetNet toy example

In [ ]:
from transformers import ProphetNetTokenizer, ProphetNetForConditionalGeneration

In [ ]:
tokenizer = ProphetNetTokenizer.from_pretrained('microsoft/prophetnet-large-uncased')
model = ProphetNetForConditionalGeneration.from_pretrained('microsoft/prophetnet-large-uncased')

In [ ]:
test_text = "An American woman died aboard a cruise ship that docked at Rio de Janeiro on Tuesday, the same ship on which 86 passengers previously fell ill, according to the state-run Brazilian news agency."

In [ ]:
input_ids = tokenizer(test_text, return_tensors="pt").input_ids  # Batch size 1

In [ ]:
#decoder_input_ids = tokenizer("An American woman", return_tensors="pt").input_ids  # Batch size 1

In [ ]:
outputs = model.generate(input_ids=input_ids, num_beams=4, max_length=30, early_stopping=True)

In [ ]:
print([tokenizer.decode(token, skip_special_tokens=True, clean_up_tokenization_spaces=False) for token in outputs])

['according to the state - run brazilian news agency , the state run news agency .']


# ProphetNet fine tuning

In [ ]:
from transformers import ProphetNetTokenizer, ProphetNetForConditionalGeneration

In [ ]:
tokenizer = ProphetNetTokenizer.from_pretrained('microsoft/prophetnet-large-uncased')
model = ProphetNetForConditionalGeneration.from_pretrained('microsoft/prophetnet-large-uncased')

In [ ]:
model.cuda()
'block print'

'block print'

In [ ]:
batch_size = 16  
encoder_max_length = 512
decoder_max_length = 128

In [ ]:
pwd

'/content/drive/My Drive/Inter_IIT_Headline_Generation/HeadlineGen/models/ProphetNet'

In [ ]:
RAW_DATA_PATH = "../../raw_data/Development Data/dev_data_article.xlsx"

In [ ]:
df = po.read_excel(RAW_DATA_PATH)

In [ ]:
df

,Text_ID,Text,Headline,Mobile_Tech_Flag
0,article_0001,Digitisation is one of the key buzzwords in th...,Pakistan’s digital landscape post-Covid,0
1,article_0002,Increase in tolerance limit up to 120 per cent...,Affordable housing gets shot in the arm,0
2,article_0003,Home > News > World Sports News\n\nJonas Lossl...,Jonas Lossl leaves Everton to rejoin first clu...,0
3,article_0004,"Source: Agfax.com\n\nBy Keith Brown, DTN Contr...",DTN Cotton Closing: Cotton Higher on Commoditi...,0
4,article_0005,"United Nations, Feb 4: The Serum Institute of ...","Serum Institute of India, UNICEF enter into lo...",0
...,...,...,...,...
3995,article_3996,All Hindi Bloggers List with All Detail\nMay 1...,All Hindi Bloggers List with All Detail,0
3996,article_3997,"Amazon Flex se Part Time me Earn kare ₹15,000 ...","Amazon Flex se Part Time me Earn kare ₹15,000 ...",0
3997,article_3998,Bad Link Building Strategies se Kaise Bachaye ...,Bad Link Building Strategies se Kaise Bachaye ...,0
3998,article_3999,Badiya Comment karte Samay Rakhe in Baat ka Dh...,Badiya Comment karte Samay Rakhe in Baat ka Dh...,0


In [ ]:
df['Text'][:20]

0     Digitisation is one of the key buzzwords in th...
1     Increase in tolerance limit up to 120 per cent...
2     Home > News > World Sports News\n\nJonas Lossl...
3     Source: Agfax.com\n\nBy Keith Brown, DTN Contr...
4     United Nations, Feb 4: The Serum Institute of ...
5     The causal effects of long-term PM2.5 exposure...
6     Poverty-centred local adaptation in Mozambique...
7     Assessing physical risks from climate change: ...
8     Home > News > Health News\n\nDiabetes during p...
9     Home > News > World News\n\nBanks closed, ATMs...
10    Not risking it: Children and youth in climate ...
11    Climate risk country profile: Nepal\n\nThis pu...
12    Climate risk country profile: Maldives\n\nThis...
13    All study data was collected between May and O...
14    A few days before the budget, the Controller G...
15    blackpink Jennie Rose Lisa Jisoo\n\nBack in 20...
16    \n\nImage: Jussi Nukari / Lehtikuva\n\n\n\n\n\...
17    When the weight of history, or the need to

In [ ]:
df['Headline'][:20]

0               Pakistan’s digital landscape post-Covid
1               Affordable housing gets shot in the arm
2     Jonas Lossl leaves Everton to rejoin first clu...
3     DTN Cotton Closing: Cotton Higher on Commoditi...
4     Serum Institute of India, UNICEF enter into lo...
5     The causal effects of long-term PM2.5 exposure...
6        Poverty-centred local adaptation in Mozambique
7     Assessing physical risks from climate change: ...
8     Diabetes during pregnancy linked to heart dise...
9     Banks closed, ATMs shut down in Myanmar follow...
10    Not risking it: Children and youth in climate ...
11                  Climate risk country profile: Nepal
12               Climate risk country profile: Maldives
13    Reinfection by COVID-19 in healthy young adult...
14    Sitharaman has lived up to her word on a ‘neve...
15    Spotify Launches in South Korea: ‘You’re Going...
16    Whether older people will receive the AstraZen...
17     A post-ideology budget: It’s bold and wel

In [ ]:
train_article_encodings = tokenizer(df['Text'][:10].tolist(), truncation=True, padding=True)
train_headline_encodings = tokenizer(df['Headline'][:10].tolist(), truncation=True, padding=True)

val_article_encodings = tokenizer(df['Text'][10:20].tolist(), truncation=True, padding=True)
val_headline_encodings = tokenizer(df['Headline'][10:20].tolist(), truncation=True, padding=True)

In [ ]:
device='cuda'

In [ ]:
class ArticleDataset(Dataset):
    def __init__(self, article_encodings, headline_encodings):
        self.article_encodings = article_encodings
        self.headline_encodings = headline_encodings

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]).to(device) for key, val in self.article_encodings.items() if key!='token_type_ids'}
        for dec_key, dec_val in self.headline_encodings.items():
            if dec_key!='token_type_ids':
              item['decoder_'+dec_key] = torch.tensor(dec_val[idx]).to(device)
          
        item['labels'] = torch.tensor(self.headline_encodings['input_ids'][idx]).to(device) # decoder_input_ids

        return item

    def __len__(self):
        return len(self.article_encodings)

In [ ]:
train_dataset = ArticleDataset(train_article_encodings, train_headline_encodings)
val_dataset = ArticleDataset(val_article_encodings, val_headline_encodings)

In [ ]:
train_dataset[0].keys()

dict_keys(['input_ids', 'attention_mask', 'decoder_input_ids', 'decoder_attention_mask', 'labels'])

In [ ]:
BATCH_SIZE=1
gradient_accumulation_steps=1
weight_decay=0
epsilon=1e-8
learning_rate=2e-5
number_of_warmup_steps=0
num_train_epochs=1
max_grad_norm=1

In [ ]:
#train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE)

In [ ]:
t_total = len(df) // gradient_accumulation_steps * 3

In [ ]:
optimizer = AdamW(model.parameters(), lr=learning_rate, eps=epsilon)
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=number_of_warmup_steps, num_training_steps=t_total)

In [ ]:
device='cuda'

In [ ]:
model.zero_grad()
global_step = 0
tr_loss = 0
for i in tqdm(range(num_train_epochs)):
  for i in tqdm(range(len(df))):
    model.train()
    
    article_encoding = tokenizer(df.iloc[i]['Text'], return_tensors="pt", max_length=512, truncation=True, padding='max_length')
    headline_encoding = tokenizer(df.iloc[i]['Headline'], return_tensors="pt", max_length=512, truncation=True, padding='max_length')

    labels = headline_encoding['input_ids'].to(device)
    labels = [[-100 if token == tokenizer.pad_token_id else token for token in l] for l in labels]

    outputs = model(input_ids=article_encoding['input_ids'].to(device), attention_mask=article_encoding['attention_mask'].to(device), decoder_input_ids=headline_encoding['input_ids'].to(device), decoder_attention_mask=headline_encoding['attention_mask'].to(device), labels=labels)

    loss = outputs[0]  
    loss.backward() 
    
    torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
    tr_loss += loss.item()
    
    if (i + 1) % gradient_accumulation_steps == 0:
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        model.zero_grad()
        global_step += 1
  
  # Save Model
  torch.save(model, 'saved_models/prophet_net_epoch{}.pt'.format(i))

RuntimeError: ignored